In [ ]:
import numpy as np
import xarray as xr
from numpy.polynomial import legendre as L

import os
import rasterio
from rasterio.transform import Affine

In [ ]:
import numpy as np
from math import pi
from scipy.ndimage import map_coordinates
import imageio.v3 as iio  # ou: from PIL import Image; np.array(Image.open(...), dtype=np.float64)

def legendreN(nmax, t):
    """
    Polinômios de Legendre (ordem 0) de 0..nmax avaliados em t (1D).
    Retorna array shape (len(t), nmax+1), col j = P_j(t).
    """
    t = np.asarray(t, dtype=np.float64)
    m = t.size
    Pn = np.zeros((m, nmax + 1), dtype=np.float64)
    Pn[:, 0] = 1.0
    if nmax >= 1:
        Pn[:, 1] = t
    for n in range(2, nmax + 1):
        # P_n = ((2n-1)/n) * t * P_{n-1} + ((1-n)/n) * P_{n-2}
        Pn[:, n] = ((2*n - 1)/n) * t * Pn[:, n-1] + ((1 - n)/n) * Pn[:, n-2]
    return Pn

def helmert_stokes_residual_geoid(
    tif_path="Grade_AG_res_Poly9.tif",
    resolucao_graus=0.05,
    raio_integracao_graus=1.0,
    grau_modificacao=300,
    latN=-22.47500000000000141 - 0.05/2,   # já com ajuste do centro do pixel
    lonW=-54.5249999999999986 + 0.05/2,     # idem
):
    # -----------------------------
    # Importando a grade de anomalias residuais (m^2/s^2) e convertendo p/ double
    # -----------------------------
    Ag_res = iio.imread(tif_path).astype(np.float64) * 1e-5
    lin, col = Ag_res.shape

    # -----------------------------
    # Grade completa (área de integração) em graus
    # -----------------------------
    latS = latN - (lin - 1) * resolucao_graus
    lonE = lonW + (col - 1) * resolucao_graus

    # Vetores de coordenadas (em graus)
    lat_vals_deg = latN - np.arange(lin) * resolucao_graus
    lon_vals_deg = lonW + np.arange(col) * resolucao_graus

    # Grade 1D de pontos de integração (em rad)
    grade_lat = np.deg2rad(lat_vals_deg)[:, None] * np.ones((1, col))
    grade_long = np.deg2rad(lon_vals_deg)[None, :] * np.ones((lin, 1))

    latQ = grade_lat.ravel()
    longQ = grade_long.ravel()
    AgQ = Ag_res.ravel()
    sin_latQ = np.sin(latQ)
    cos_latQ = np.cos(latQ)

    # -----------------------------
    # Grade de pontos de cálculo (recortada pelo raio de integração)
    # -----------------------------
    latP_N = latN - raio_integracao_graus
    lonP_W = lonW + raio_integracao_graus
    latP_S = latS + raio_integracao_graus
    lonP_E = lonE - raio_integracao_graus

    lin_modelo = int(np.floor((latP_N - latP_S) / resolucao_graus) + 1)
    col_modelo = int(np.floor((lonP_E - lonP_W) / resolucao_graus) + 1)

    latP_grid_deg = (latP_N - np.arange(lin_modelo) * resolucao_graus)[:, None] * np.ones((1, col_modelo))
    lonP_grid_deg = (lonP_W + np.arange(col_modelo) * resolucao_graus)[None, :] * np.ones((lin_modelo, 1))

    latP = np.deg2rad(latP_grid_deg).ravel()
    longP = np.deg2rad(lonP_grid_deg).ravel()

    # -----------------------------
    # Interpolação bicúbica de AgP nos pontos P (em rad → usa deg p/ índices)
    # -----------------------------
    latP_deg = np.rad2deg(latP)
    longP_deg = np.rad2deg(longP)

    # linhas/colunas fracionárias relativas a (latN, lonW)
    rows = (latN - latP_deg) / resolucao_graus
    cols = (longP_deg - lonW) / resolucao_graus
    AgP = map_coordinates(Ag_res, [rows, cols], order=3, mode="nearest")

    # -----------------------------
    # Gravidade normal (G1) no elipsoide, gama0(latP), em m/s^2
    # -----------------------------
    sin_latP = np.sin(latP)
    gama0 = 9.7803267715 * (
        1
        + 0.0052790414 * (sin_latP**2)
        + 0.0000232718 * (sin_latP**4)
        + 0.0000001262 * (sin_latP**6)
        + 0.0000000007 * (sin_latP**8)
    )

    # -----------------------------
    # Constantes
    # -----------------------------
    res_rad = np.deg2rad(resolucao_graus)
    raio_integ_rad = np.deg2rad(raio_integracao_graus)
    R = 6_371_000.0  # raio da esfera de mesmo volume (m)
    res_rad_div_2 = res_rad / 2
    cte = 4 * pi * R * gama0
    n_mod = np.arange(2, grau_modificacao + 1, dtype=np.float64)  # 2..N
    cte_mod = (2 * n_mod + 1) / (n_mod - 1)                       # vetor (N-1,)

    # -----------------------------
    # Áreas dos pixels (integração) — por ponto Q
    # -----------------------------
    Ak = (R**2) * res_rad * (np.sin(latQ + res_rad_div_2) - np.sin(latQ - res_rad_div_2))
    Ak = np.abs(Ak)

    # -----------------------------
    # Contribuição da zona mais interna (Ni)
    # -----------------------------
    Ni = (R / gama0) * np.sqrt((np.cos(latP) * res_rad**2) / pi) * AgP  # m

    # -----------------------------
    # Integração Helmert–Stokes
    # -----------------------------
    N_res = np.zeros(latP.size, dtype=np.float64)
    cos_latP = np.cos(latP)

    # máscara/índices auxiliares para achar o “ponto singular”
    latQ_deg = np.rad2deg(latQ)
    longQ_deg = np.rad2deg(longQ)

    for p in range(latP.size):
        # Ângulo esférico entre P (latP[p], longP[p]) e todos Q
        cos_psi = sin_latQ * sin_latP[p] + cos_latQ * cos_latP[p] * np.cos(longQ - longP[p])
        cos_psi = np.clip(cos_psi, -1.0, 1.0)
        psi = np.arccos(cos_psi)

        # Filtra fora o que está além do raio de integração
        mask = psi <= raio_integ_rad
        if not np.any(mask):
            N_res[p] = Ni[p]
            continue

        psi_i = psi[mask]
        cos_psi_i = cos_psi[mask]
        latQ_i = latQ[mask]
        longQ_i = longQ[mask]
        AgQ_i = AgQ[mask]
        Ak_i = Ak[mask]

        # Localiza o pixel Q mais próximo de P para remover a singularidade
        # (aproxima a “coincidência” por mínimos em lat e lon)
        # usa graus para minimizar erro de arredondamento igual ao MATLAB
        L1 = np.where(np.abs(np.rad2deg(latP[p]) - latQ_deg[mask]) ==
                      np.min(np.abs(np.rad2deg(latP[p]) - latQ_deg[mask])))[0]
        L2 = np.where(np.abs(np.rad2deg(longP[p]) - longQ_deg[mask]) ==
                      np.min(np.abs(np.rad2deg(longP[p]) - longQ_deg[mask])))[0]
        # interseção
        pos_p_idx = np.intersect1d(L1, L2, assume_unique=False)

        # Função de Stokes modificada (Wang & Gore, 1969)
        # s^2 (meia-corda esférica)
        s2 = (
            np.sin((latP[p] - latQ_i) / 2.0) ** 2
            + (np.sin((longP[p] - longQ_i) / 2.0) ** 2) * cos_latP[p] * np.cos(latQ_i)
        )
        # proteção contra log(0)
        s2 = np.clip(s2, 1e-30, None)
        s = np.sqrt(s2)
        S_psi = 1.0 / s - 4.0 - 6.0 * s + 10.0 * s2 - (3.0 - 6.0 * s2) * np.log(s2 + s)

        # remove termo singular (no ponto de cálculo)
        if pos_p_idx.size > 0:
            S_psi[pos_p_idx] = np.nan

        # Série harmônica esférica de modificação (somatório n=2..N)
        Pn = legendreN(grau_modificacao, cos_psi_i)     # (m, nmax+1)
        # descarta n=0,1 → colunas 2..N (índice 2:)
        # S_psi_SH(j) = sum_{n=2..N} cte_mod[n-2] * P_n(cos_psi_j)
        S_psi_SH = Pn[:, 2:] @ cte_mod                  # (m,)

        termo = (S_psi - S_psi_SH) * Ak_i * AgQ_i / cte[p]
        N_res[p] = np.nansum(termo) + Ni[p]

    # Resultado: lat, lon, N (m)
    Resultado = np.column_stack([latP_deg, longP_deg, N_res])
    # também devolvo a grade reshapeada, se quiser salvar como GeoTIFF depois
    N_grid = N_res.reshape(lin_modelo, col_modelo)
    lat_grid = latP_grid_deg
    lon_grid = lonP_grid_deg
    return Resultado, (N_grid, lat_grid, lon_grid)

# Exemplo de uso:
# Resultado, (N_grid, lat_grid, lon_grid) = helmert_stokes_residual_geoid(
#     "Grade_AG_res_Poly9.tif",
#     resolucao_graus=0.05,
#     raio_integracao_graus=1.0,
#     grau_modificacao=300,
# )
# print(Resultado.shape)  # (lin_modelo*col_modelo, 3)
